In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
import datetime as dt
import os
import seaborn as sns

import plotly.express as px

import random
import string

import polars as pl
import polars.selectors as cs

# Read files

In [ ]:
from nb_vars import FOLDER_INTERMEDIATE, CUTOFF_SPIKES_HZ

In [ ]:
df_peaks = pl.read_csv(f'{FOLDER_INTERMEDIATE}/df_peaks_full.csv')
df_peaks = df_peaks.to_pandas()

In [ ]:
# %%script true
# Plot each well
df_peaks_sub = df_peaks[(df_peaks['time_exp'] == '10 min') & 
                        (df_peaks['well'] == 'D4')]

fig = plt.figure(figsize=(30, 6))

y = [(int(i[0]) - 1) * 4 + (int(i[1]) - 1) for i in df_peaks_sub['electrode'].astype(str).values]
x =  df_peaks_sub['time'].values 

fig = px.scatter(x=x, y=y, opacity=0.35)
fig.update_yaxes(tickvals=np.arange(16), ticktext=[f'{i//4 + 1}{i%4 + 1}' for i in np.arange(16)])

## Plot the activity per minute per electrode

In [ ]:
# !!! WARNING: ONCE YOU SET UP THIS VARIABLE, CHANGE IT IN THE nb_vars.py FILE!!!!

CUTOFF_SPIKES_HZ = 1

In [ ]:
df_list_wells, df_list_activity = [], []

for well in df_peaks['well'].drop_duplicates().sort_values().values:
    list_electrodes_refuse = []

    df_well = df_peaks[df_peaks['well'] == well]

    for time_exp in sorted(set(df_well['time_exp'].values)):
        df_well_treatment = df_well[df_well['time_exp'] == time_exp]
        list_counts = df_well_treatment.groupby('electrode').count()['time'].sort_values() / (df_well['time'].max() - df_well['time'].min()) # * 60 para hacerlo relativo a minuto

        df_list_wells += ['\n'.join([well, time_exp])] * len(list_counts)
        df_list_activity += list_counts.values.tolist()

df_activity = {'well': df_list_wells, 'activity': df_list_activity}

fig = plt.figure(figsize=(100, 6))
sns.stripplot(data=df_activity, x='well', y='activity')
plt.xticks(rotation=90)
plt.plot(plt.gca().get_xlim(), [CUTOFF_SPIKES_HZ, CUTOFF_SPIKES_HZ], c='#bc0000')

**There are two wells that have 0 activity: A2 (HEK PLKO) and D4 (ASTROS 219); and C6 (ASTROS PLKO) was removed after the 1Hz cutoff.**

In [ ]:
# MAKE A LIST OF ELECTRODES TO REFUSE
# Since each condition has a different set of wells, we only need to encode the well info

# We are going to refuse electrodes with less than XX spikes PER MINUTE.

dict_electrode_refuse = {}


for well in sorted(set(df_peaks['well'].values)):
    list_electrodes_refuse = []

    df_well = df_peaks[df_peaks['well'] == well]

    for time_exp in sorted(set(df_well['time_exp'].values)):
        df_well_time_exp = df_well[df_well['time_exp'] == time_exp]

        list_counts = df_well_time_exp.groupby('electrode').count()['time'].sort_values() / (df_well['time'].max() - df_well['time'].min()) # * 60 para hacerlo relativo a minuto
        list_electrodes = list_counts[list_counts < CUTOFF_SPIKES_HZ].index.tolist()

        list_electrodes_refuse += list_electrodes

    set_electrodes_refuse = sorted(list(set(list_electrodes_refuse)))
    dict_electrode_refuse[well] = set_electrodes_refuse
        

print(dict_electrode_refuse)


list_remove_idx = []

for well, list_electrodes in dict_electrode_refuse.items():
    for electrode in list_electrodes:
        df_sub = df_peaks[(df_peaks['well'] == well) & (df_peaks['electrode'] == int(electrode))]
        list_remove_idx += df_sub.index.tolist()

list_remove_idx_bool = np.ones(len(df_peaks)).astype(bool)
list_remove_idx_bool[list_remove_idx] = False

df_peaks_sub_filter = df_peaks.loc[list_remove_idx_bool]

In [ ]:
df_peaks

In [ ]:
df_peaks_sub_filter

In [ ]:
# FASTER TO SAVE WITH POLARS
pl.DataFrame(df_peaks_sub_filter).write_csv(f'{FOLDER_INTERMEDIATE}/df_peaks_full_freq_{CUTOFF_SPIKES_HZ}.csv')

In [ ]:
with open(f'{FOLDER_INTERMEDIATE}/freq_{CUTOFF_SPIKES_HZ}_discarded_electrodes.txt', 'w') as file:
    for well, elecs in dict_electrode_refuse.items():
        file.write(f"{well}: {', '.join([str(i) for i in elecs])}\n")